In [3]:
import math
def deg2rad(deg):
    return deg * math.pi / 180

In [2]:
a = 6378137.0
b = 6356752.3


R = a

In [4]:
def toMerc(lng, lat):
    x = R * deg2rad(lng)
    y = R * math.log(math.tan(math.pi/4 + deg2rad(lat)/2))

    return x,y

In [5]:
toMerc(-180, 0)

(-20037508.342789244, -7.081154551613622e-10)

In [6]:
toMerc(180, 0)

(20037508.342789244, -7.081154551613622e-10)

In [11]:
q = toMerc(180, 0)[0] - toMerc(-180, 0)[0] # Erdumfang

In [13]:
q / (2*math.pi) # ergibt wieder den Erdradius

6378137.000000001

In [16]:
from pyproj import Transformer

wgs84merc = Transformer.from_crs(4326, 3857, always_xy=True)

In [17]:
wgs84merc.transform(-180, 0)

(-20037508.342789244, 0.0)

In [18]:
wgs84merc.transform(180, 0)

(20037508.342789244, 0.0)

In [19]:
mercwgs84 = Transformer.from_crs(3857, 4326, always_xy=True)

In [20]:
mercwgs84.transform(-20037508.342789244, 0.0)

(-180.0, 0.0)

In [21]:
mercwgs84.transform(-20037508.342789244, -20037508.342789244)

(-180.0, -85.0511287798066)

In [22]:
mercwgs84.transform(20037508.342789244, 20037508.342789244)

(180.0, 85.0511287798066)

In [23]:
def QuadKeyToNormalizedMercatorCoord(key):
    zoomlevels = len(key)
    x = 0
    y = 0
    scale = 1.0
    for i in range(0,zoomlevels):
        scale /= 2.0
        if key[i] == "0":
            y += scale
        elif key[i] == "1":
            x += scale
            y += scale
        elif key[i] == "2":
            pass
        elif key[i] == "3":
            x += scale

    return [ 2*x-1, 2*y-1 , 2*(x + scale)-1, 2*(y + scale)-1]


QuadKeyToNormalizedMercatorCoord("1")

[0.0, 0.0, 1.0, 1.0]